In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import gc
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.optimizer_v2 import adam

In [ ]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [ ]:
image_path = "./dataset_f+r20/images.npy"
emotion_path = "./dataset_f+r20/emotions_multi.npy"

images = np.load(image_path)
images = tf.convert_to_tensor(images)
images = tf.image.grayscale_to_rgb(images)

emotions = np.load(emotion_path)
emotions = tf.convert_to_tensor(emotions)
    
training_samples = 28317 * 4
validation_samples = 3541 * 4
training_size = training_samples + validation_samples
    
training_images = images[:training_size]
test_images = images[training_size:]
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

In [ ]:

image_path_affectnet = "./AffectNet/images.npy"
emotion_path_affectnet = "./AffectNet/emotions.npy"

images_affectnet = np.load(image_path_affectnet)
images_affectnet = tf.convert_to_tensor(images_affectnet)

emotions_affectnet = np.load(emotion_path_affectnet)
emotions_affectnet = tf.convert_to_tensor(emotions_affectnet)


In [ ]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [ ]:
# affectnet_pretrain=1: AffectNet pretrained
def train(model, learning_rate, loss, num_epochs, batch_size, affectnet_pretrain):
    
    if affectnet_pretrain == 1:
        model.compile(optimizer=adam.Adam(learning_rate=1e-5), 
                  loss=loss, 
                  metrics = [model_acc])
        model.fit(x=images_affectnet,
                  y=emotions_affectnet,
                  batch_size=batch_size,
                  epochs=num_epochs)
        
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    history = model.fit(x=training_images,
                        y=training_emotions,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=(test_images, test_emotions))
    del model
    gc.collect()
    return history

In [ ]:
def create_model(base_model):
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
if not os.path.isdir('./results/'):
    os.mkdir('./results/')

learning_rate = 1e-4
num_epochs = 40
batch_size = 32
loss = losses.MeanSquaredError()
for i in range(2,-1,-1):
    if i == 0: # none
        base_model = vgg16.VGG16(include_top=False, weights=None, input_shape=(48,48,3))
        model = create_model(base_model)
        history_save_path = './results/none.txt'
        history = train(model, learning_rate, loss, num_epochs, batch_size, 0)
    if i == 1: # AffectNet
        base_model = vgg16.VGG16(include_top=False, weights=None, input_shape=(48,48,3))
        model = create_model(base_model)
        history_save_path = './results/AN.txt'
        history = train(model, learning_rate, loss, num_epochs, batch_size, 1)
    if i == 2: # ImageNet-AffectNet
        base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(48,48,3))
        model = create_model(base_model)
        history_save_path = './results/IN-AN.txt'
        history = train(model, learning_rate, loss, num_epochs, batch_size, 1)

    with open(history_save_path, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    del model, history
    gc.collect()

In [ ]:
model.fit(x=training_images,
                        y=training_emotions,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=(test_images, test_emotions))

In [ ]:
# model comparison
epochs = range(1, 1+num_epochs)
fig = plt.figure()

with open('./results/history_base+flip+rotate20.txt') as file_pi:
    hist = pickle.load(file_pi)
plt.plot(epochs, hist['val_model_acc'], label = 'IN')

with open('./results/none.txt') as file_pi:
    hist = pickle.load(file_pi)
plt.plot(epochs, hist['val_model_acc'], label = 'None')

with open('./results/AN.txt') as file_pi:
    hist = pickle.load(file_pi)
plt.plot(epochs, hist['val_model_acc'], label = 'AN')

with open('./results/IN-AN.txt') as file_pi:
    hist = pickle.load(file_pi)
plt.plot(epochs, hist['val_model_acc'], label = 'IN-AN')

plt.legend()
plt.show()
# save plot
fig.savefig('./plots/pretrained.png')